In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta as rd
import time
import math
from sklearn.metrics import mean_squared_error

In [2]:
# stocks data csv read
df = pd.read_csv('data.csv')
df = df.set_index('Date')

# s&p data csv read
df_sp = pd.read_csv('sp500.csv')
df_sp = df_sp.set_index('Date')

# stocks data csv read for partial replication
df_reduce = pd.read_csv('data.csv')
df_reduce = df_reduce.set_index('Date')

In [3]:
def date_slicer(df, start, duration, rebalancing_period=0):
    start = str(datetime.strptime(start, '%Y-%m-%d').date() + rd(months=rebalancing_period))
    end = str(datetime.strptime(start, '%Y-%m-%d').date() + rd(months=duration) - rd(days=1))
    return df.loc[start:end]

In [4]:
def data_process(df):
    df = df.pct_change()
    df = df.tail(-1)
    df = df + 1
    df = df.cumprod()
    df = df - 1
    df = df.iloc[-1,:]
    df = df.to_numpy()
    df = torch.from_numpy(df).type(torch.Tensor)
    return df

In [5]:
def daily_change(df):
    df = df.pct_change()
    df = df.tail(-1)
    return df

In [6]:
def daily_return(df):
    df = df.pct_change()
    df = df.tail(-1)
    df = df + 1
    return df

In [7]:
def index_finder(df):
    df = df.pct_change()
    df = df.tail(-1)
    df = df + 1
    df = df.cumprod()
    df = df - 1
    df = df.iloc[-1,:]
    return df

In [8]:
stocks_index = index_finder(df).index

In [9]:
# shallow nnf biuld
class shallow_NNF(nn.Module):
    def __init__(self, input_dim, hidden_size, num_classes):
        super(shallow_NNF, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
        
    def reset_parameters(self):
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.softmax(self.fc2(out))
        weights = out
        cumulative_change = sum(out * x)
        return cumulative_change, weights

In [10]:
# shallow nnf biuld
class shallow_NNF_partial(nn.Module):
    def __init__(self, input_dim, hidden_size, num_classes):
        super(shallow_NNF_partial, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
        
    def reset_parameters(self):
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.softmax(self.fc2(out))
        weights = out
        cumulative_change = sum(out * x)
        return cumulative_change, weights

In [11]:
# deep nnf build
class deep_NNF(nn.Module):
    def __init__(self, input_dim, hidden_size1, hidden_size2, hidden_size3,
                 hidden_size4, hidden_size5, num_classes, dropout_p = 0.2):
        super(deep_NNF, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, hidden_size4)
        self.fc5 = nn.Linear(hidden_size4, hidden_size5)
        self.fc6 = nn.Linear(hidden_size5, num_classes)
    
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_p)
        self.softmax = nn.Softmax(dim=0)
        
    def reset_parameters(self):
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        self.fc3.reset_parameters()
        self.fc4.reset_parameters()
        self.fc5.reset_parameters()
        self.fc6.reset_parameters()
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.dropout(out)
        out = self.relu(self.fc2(out))
        out = self.dropout(out)
        out = self.relu(self.fc3(out))
        out = self.dropout(out)
        out = self.relu(self.fc4(out))
        out = self.dropout(out)
        out = self.relu(self.fc5(out))
        out = self.softmax(self.fc6(out))
        weights = out
        cumulative_change = sum(out * x)
        return cumulative_change, weights

In [12]:
class deep_NNF_partial(nn.Module):
    def __init__(self, input_dim, hidden_size1, hidden_size2, hidden_size3,
                 hidden_size4, hidden_size5, num_classes, dropout_p = 0.2):
        super(deep_NNF_partial, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, hidden_size4)
        self.fc5 = nn.Linear(hidden_size4, hidden_size5)
        self.fc6 = nn.Linear(hidden_size5, num_classes)
    
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_p)
        self.softmax = nn.Softmax(dim=0)
        
    def reset_parameters(self):
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        self.fc3.reset_parameters()
        self.fc4.reset_parameters()
        self.fc5.reset_parameters()
        self.fc6.reset_parameters()
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.dropout(out)
        out = self.relu(self.fc2(out))
        out = self.dropout(out)
        out = self.relu(self.fc3(out))
        out = self.dropout(out)
        out = self.relu(self.fc4(out))
        out = self.dropout(out)
        out = self.relu(self.fc5(out))
        out = self.softmax(self.fc6(out))
        weights = out
        cumulative_change = sum(out * x)
        return cumulative_change, weights

In [13]:
# 1/N model build
class equal_w_model():
    def __init__(self, df):
        self.df = df
        self.performance()
        
    def performance(self):
        self.df = np.array(self.df)
        weights = np.ones((len(self.df), 1)) * (1/len(self.df))
        cumulative_change = sum(np.multiply(weights, self.df.reshape(-1,1)))
        return cumulative_change, weights.reshape(-1)

In [14]:
# rebalancing period = one or three months
rbp = 1

# number of companies in the partial portfolio
partial_num = 200

# epochs
num_epochs = 100

In [15]:
# shallow_nnf hyperparameters
input_dim = 471
hidden_size = 471
num_classes = 471
lr = 1e-1

In [16]:
# shallow nnf tune
shallow_NNF = shallow_NNF(input_dim=input_dim, hidden_size=hidden_size, num_classes=num_classes)
shallow_NNF_loss_fun = torch.nn.MSELoss(reduction='mean')
shallow_NNF_optimizer = torch.optim.Adam(shallow_NNF.parameters(), lr=lr)

In [17]:
# shallow nnf partial tune
shallow_NNF_partial = shallow_NNF_partial(input_dim=partial_num, hidden_size=hidden_size, num_classes=partial_num)
shallow_NNF_partial_loss_fun = torch.nn.MSELoss(reduction='mean')
shallow_NNF_partial_optimizer = torch.optim.Adam(shallow_NNF_partial.parameters(), lr=lr)

In [18]:
# deep_nnf hyperparameters
input_dim = 471
hidden_size1 = 471
hidden_size2 = 471
hidden_size3 = 471
hidden_size4 = 471
hidden_size5 = 471
num_classes = 471
lr = 1e-3
dropout_p = 0.2

In [19]:
# deep nnf tune
deep_NNF = deep_NNF(input_dim=input_dim, hidden_size1=hidden_size1, hidden_size2=hidden_size2, 
                    hidden_size3=hidden_size3, hidden_size4=hidden_size4, hidden_size5=hidden_size5,
                    num_classes=num_classes)
deep_NNF_loss_fun = torch.nn.MSELoss(reduction='mean')
deep_NNF_optimizer = torch.optim.Adam(deep_NNF.parameters(), lr=lr)

In [20]:
# deep nnf partial tune
deep_NNF_partial = deep_NNF_partial(input_dim=partial_num, hidden_size1=hidden_size1, hidden_size2=hidden_size2, 
                    hidden_size3=hidden_size3, hidden_size4=hidden_size4, hidden_size5=hidden_size5,
                    num_classes=partial_num)
deep_NNF_partial_loss_fun = torch.nn.MSELoss(reduction='mean')
deep_NNF_partial_optimizer = torch.optim.Adam(deep_NNF_partial.parameters(), lr=lr)

In [21]:
# RMSE
def RMSE(x, y, weights):
    temp = 0
    for i in range(len(x)):
        temp += (sum(x.iloc[i] * weights) - y.iloc[i]) ** 2
    return math.sqrt(temp/len(x))

In [22]:
# MEAN
def MEAN(x, weights):
    temp = []
    for i in range(len(x)):
        temp.append(sum(x.iloc[i] * weights))
    temp = np.array(temp)
    return temp.mean()

In [23]:
# Volatility
def VOL(x, weights):
    temp = []
    for i in range(len(x)):
        temp.append(sum(x.iloc[i] * weights))
    temp = np.array(temp)
    return temp.std()

In [24]:
def valid_fun(x_valid, i, model):
    x_change = daily_change(date_slicer(df_reduce, '2017-07-01', 6, i))
    y_change = daily_change(date_slicer(df_sp, '2017-07-01', 6, i))
    # x_return = daily_return(date_slicer(df, '2017-07-01', 6, i))
    # y_return = daily_return(date_slicer(df_sp, '2017-07-01', 6, i))
    
    if model == equal_w_model:
        weights = model(x_valid).performance()[1]
    else:
        weights = np.array(model(x_valid)[1].detach())
    
    valid_rmse = RMSE(x_change, y_change, weights)
    # valid_mean = MEAN(x_return, weights)
    # valid_vol  = VOL(x_return, weights)
    
    print(f'Validation RMSE: {valid_rmse}')
    # print(f'Validation MEAN: {valid_mean}')
    # print(f'Validation VOL: {valid_vol}')
    
    return valid_rmse

In [25]:
def test_fun(x_test, i, model):
    x_change = daily_change(date_slicer(df_reduce, '2018-01-01', 6, i))
    y_change = daily_change(date_slicer(df_sp, '2018-01-01', 6, i))
    x_return = daily_return(date_slicer(df_reduce, '2018-01-01', 6, i))
    y_return = daily_return(date_slicer(df_sp, '2018-01-01', 6, i))
    
    if model == equal_w_model:
        weights = model(x_test).performance()[1]
    else:
        weights = np.array(model(x_test)[1].detach())
    
    test_rmse = RMSE(x_change, y_change, weights)
    test_mean = MEAN(x_return, weights)
    test_vol  = VOL(x_return, weights)
    test_dic = {'RMSE': test_rmse, 'MEAN': test_mean, 'VOL': test_vol}
    
    print(f'Test RMSE: {test_rmse}')
    print(f'Test MEAN: {test_mean}')
    print(f'Test VOL: {test_vol}')
    
    return test_dic

### **Deep NNF Training**

In [26]:
# deep nnf training function
def train_deep_nnf(x_train, y_train, i):
    start_time_deep_nnf = time.time()
    print(f'\nDeep NNF Training & Results for model {i+1} (Full Reblication) :')
    
    for epoch in range(num_epochs):
        y_train_pred = deep_NNF(x_train)[0]
        loss_deep_nnf = deep_NNF_loss_fun(y_train_pred, y_train)
        if epoch == 0 or epoch == num_epochs-1:
            weights = np.array(deep_NNF(x_train)[1].detach())
            print(f'Epoch {epoch+1} of {num_epochs} | MSE: {loss_deep_nnf.item()}')
        deep_NNF_optimizer.zero_grad()
        loss_deep_nnf.backward()
        deep_NNF_optimizer.step()
        
    training_time = format(time.time()-start_time_deep_nnf, '0.2f')
    print(f'Training time: {training_time}')
    
    return weights

In [27]:
def train_deep_nnf_partial(x_train, y_train, i):    
    start_time_deep_nnf = time.time()
    print(f'\nDeep NNF Training & Results for model {i+1} (Partial Reblication):')
    
    for epoch in range(num_epochs):
        y_train_pred = deep_NNF_partial(x_train)[0]
        loss_deep_nnf = deep_NNF_partial_loss_fun(y_train_pred, y_train)
        if epoch == 0 or epoch == num_epochs-1:
            print(f'Epoch {epoch+1} of {num_epochs} | MSE: {loss_deep_nnf.item()}')
        deep_NNF_partial_optimizer.zero_grad()
        loss_deep_nnf.backward()
        deep_NNF_partial_optimizer.step()
        
    training_time = format(time.time()-start_time_deep_nnf, '0.2f')
    print(f'Training time: {training_time}')

In [28]:
def partial(x_train, x_valid, x_test, weights, stocks_index, num = partial_num):
    df_partial = pd.DataFrame({'x_train': x_train, 'x_valid': x_valid, 'x_test': x_test,
                               'weights': weights}, index = stocks_index)
    df_partial = df_partial.sort_values(by = ['weights'])
    out_index = df_partial.index[num:]
    df_partial = df_partial.iloc[:num]
    
    x_train = df_partial['x_train'].to_numpy()
    x_valid = df_partial['x_valid'].to_numpy()
    x_test = df_partial['x_test'].to_numpy()
    
    x_train = torch.from_numpy(x_train).type(torch.Tensor)
    x_valid = torch.from_numpy(x_valid).type(torch.Tensor)
    x_test = torch.from_numpy(x_test).type(torch.Tensor)
    
    return x_train, x_valid, x_test, out_index

In [30]:
# deep nnf
deep_nnf_valid_rmse_list = []
deep_nnf_test_results = []
out_index_history = []

for i in range(24):
    df_reduce = df.copy()    
    x_train = data_process(date_slicer(df, '2014-07-01', 36, i))
    y_train = data_process(date_slicer(df_sp, '2014-07-01', 36, i))
    x_valid = data_process(date_slicer(df, '2017-07-01', 6, i))
    y_valid = data_process(date_slicer(df_sp, '2017-07-01', 6, i))
    x_test = data_process(date_slicer(df, '2018-01-01', 1, i))
    y_test = data_process(date_slicer(df_sp, '2018-01-01', 1, i))
    weights = train_deep_nnf(x_train, y_train, i)
    deep_NNF.reset_parameters()
    x_train, x_valid, x_test, out_index = partial(x_train, x_valid, x_test, weights, stocks_index, num = partial_num)
    out_index_history.append(out_index)
    df_reduce = df_reduce.drop(out_index, axis=1)
    train_deep_nnf_partial(x_train, y_train, i)
    deep_nnf_valid_rmse_list.append(valid_fun(x_valid, i, deep_NNF_partial))
    deep_nnf_test_results.append(test_fun(x_test, i, deep_NNF_partial))
    deep_NNF_partial.reset_parameters()

print(f'\nMin Valid RMSE is: {min(deep_nnf_valid_rmse_list)} for model i = {deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))+1}')
print('Selected Model Test Results are:')
print('RMSE =', deep_nnf_test_results[deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))]['RMSE'])
print('MEAN =', deep_nnf_test_results[deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))]['MEAN'])
print('VOL =', deep_nnf_test_results[deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))]['VOL'])

deep_best_result_index = deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))


Deep NNF Training & Results for model 1 (Full Reblication) :
Epoch 1 of 100 | MSE: 0.045541681349277496
Epoch 100 of 100 | MSE: 2.3581876575917704e-06
Training time: 1.24

Deep NNF Training & Results for model 1 (Partial Reblication):
Epoch 1 of 100 | MSE: 0.41167986392974854
Epoch 100 of 100 | MSE: 3.197442310920451e-12
Training time: 1.14
Validation RMSE: 0.00182774311701721
Test RMSE: 0.0018622776235540986
Test MEAN: 1.0004426242253592
Test VOL: 0.00965754891270736

Deep NNF Training & Results for model 2 (Full Reblication) :
Epoch 1 of 100 | MSE: 0.05360008776187897
Epoch 100 of 100 | MSE: 5.021081506129121e-06
Training time: 1.33

Deep NNF Training & Results for model 2 (Partial Reblication):
Epoch 1 of 100 | MSE: 0.09134767204523087
Epoch 100 of 100 | MSE: 0.0002157684211852029
Training time: 0.80
Validation RMSE: 0.001801656428236204
Test RMSE: 0.0019803303951644495
Test MEAN: 1.0003096148429642
Test VOL: 0.010074297250498099

Deep NNF Training & Results for model 3 (Full Rebli

### **Shallow NNF Training**

In [31]:
# shallow nnf training function
def train_shallow_nnf(x_train, y_train, i):
    start_time_shallow_nnf = time.time()
    print(f'\nShallow NNF Training & Results for model {i+1}:')
    
    for epoch in range(num_epochs):
        y_train_pred = shallow_NNF(x_train)[0]
        loss_shallow_nnf = shallow_NNF_loss_fun(y_train_pred, y_train)
        if epoch == 0 or epoch == num_epochs-1:
            weights = np.array(deep_NNF(x_train)[1].detach())
            print(f'Epoch {epoch+1} of {num_epochs} | MSE: {loss_shallow_nnf.item()}')
        shallow_NNF_optimizer.zero_grad()
        loss_shallow_nnf.backward()
        shallow_NNF_optimizer.step()
        
    training_time = format(time.time()-start_time_shallow_nnf, '0.2f')
    print(f'Training time: {training_time}')
    
    return weights

In [32]:
def train_shallow_nnf_partial(x_train, y_train, i):    
    start_time_shallow_nnf = time.time()
    print(f'\nDeep NNF Training & Results for model {i+1} (Partial Reblication):')
    
    for epoch in range(num_epochs):
        y_train_pred = shallow_NNF_partial(x_train)[0]
        loss_shallow_nnf = shallow_NNF_partial_loss_fun(y_train_pred, y_train)
        if epoch == 0 or epoch == num_epochs-1:
            print(f'Epoch {epoch+1} of {num_epochs} | MSE: {loss_shallow_nnf.item()}')
        shallow_NNF_partial_optimizer.zero_grad()
        loss_shallow_nnf.backward()
        shallow_NNF_partial_optimizer.step()
        
    training_time = format(time.time()-start_time_shallow_nnf, '0.2f')
    print(f'Training time: {training_time}')

In [33]:
#shallow nnf
shallow_nnf_valid_rmse_list = []
shallow_nnf_test_results = []

for i in range(24):
    df_reduce = df.copy()
    x_train = data_process(date_slicer(df, '2014-07-01', 36, i))
    y_train = data_process(date_slicer(df_sp, '2014-07-01', 36, i))
    x_valid = data_process(date_slicer(df, '2017-07-01', 6, i))
    y_valid = data_process(date_slicer(df_sp, '2017-07-01', 6, i))
    x_test = data_process(date_slicer(df, '2018-01-01', 1, i))
    y_test = data_process(date_slicer(df_sp, '2018-01-01', 1, i))
    weights = train_shallow_nnf(x_train, y_train, i)
    shallow_NNF.reset_parameters()
    x_train, x_valid, x_test, out_index = partial(x_train, x_valid, x_test, weights, stocks_index, num = partial_num)
    df_reduce = df_reduce.drop(out_index, axis=1)
    train_shallow_nnf_partial(x_train, y_train, i)
    shallow_nnf_valid_rmse_list.append(valid_fun(x_valid, i, shallow_NNF_partial))
    shallow_nnf_test_results.append(test_fun(x_test, i, shallow_NNF_partial))
    shallow_NNF_partial.reset_parameters()

# print(f'\nMin Valid RMSE is: {min(valid_rmse_list)} for model i = {(deep_best_result_index)+1}')
print('Selected Model Test Results for model i =', (deep_best_result_index)+1, 'are: ')
print('RMSE =', shallow_nnf_test_results[(deep_best_result_index)]['RMSE'])
print('MEAN =', shallow_nnf_test_results[(deep_best_result_index)]['MEAN'])
print('VOL =', shallow_nnf_test_results[(deep_best_result_index)]['VOL'])


Shallow NNF Training & Results for model 1:
Epoch 1 of 100 | MSE: 0.04413691908121109
Epoch 100 of 100 | MSE: 0.00619890820235014
Training time: 0.79

Deep NNF Training & Results for model 1 (Partial Reblication):
Epoch 1 of 100 | MSE: 0.02401232160627842
Epoch 100 of 100 | MSE: 0.003513181349262595
Training time: 0.39
Validation RMSE: 0.014704765628696489
Test RMSE: 0.015605680154575658
Test MEAN: 0.9967217342144936
Test VOL: 0.01638193688046917

Shallow NNF Training & Results for model 2:
Epoch 1 of 100 | MSE: 0.05669238790869713
Epoch 100 of 100 | MSE: 0.025384142994880676
Training time: 1.02

Deep NNF Training & Results for model 2 (Partial Reblication):
Epoch 1 of 100 | MSE: 0.03299961984157562
Epoch 100 of 100 | MSE: 0.001196506666019559
Training time: 0.46
Validation RMSE: 0.007473235524693418
Test RMSE: 0.012615970621238187
Test MEAN: 1.000618215634368
Test VOL: 0.014506729454527854

Shallow NNF Training & Results for model 3:
Epoch 1 of 100 | MSE: 0.05106727033853531
Epoch 10

### **1/N Model**

In [34]:
equal_w_model_valid_rmse_list = []
equal_w_model_test_results = []

for i in range(24):
    df_reduce = df.copy()
    df_reduce = df_reduce.drop(out_index_history[i], axis=1)
    print(f'\nEqual Weights Model Results for model {i+1}:')
    x_train = data_process(date_slicer(df_reduce, '2014-07-01', 36, i))
    y_train = data_process(date_slicer(df_sp, '2014-07-01', 36, i))
    x_valid = data_process(date_slicer(df_reduce, '2017-07-01', 6, i))
    y_valid = data_process(date_slicer(df_sp, '2017-07-01', 6, i))
    x_test = data_process(date_slicer(df_reduce, '2018-01-01', 1, i))
    y_test = data_process(date_slicer(df_sp, '2018-01-01', 1, i))
    
    equal_w_model_valid_rmse_list.append(valid_fun(x_valid, i, equal_w_model))
    equal_w_model_test_results.append(test_fun(x_test, i, equal_w_model))
    
print('Selected Model Test Results for model i =', (deep_best_result_index)+1, 'are: ')
print('RMSE =', equal_w_model_test_results[(deep_best_result_index)]['RMSE'])
print('MEAN =', equal_w_model_test_results[(deep_best_result_index)]['MEAN'])
print('VOL =', equal_w_model_test_results[(deep_best_result_index)]['VOL'])


Equal Weights Model Results for model 1:
Validation RMSE: 0.001807070653845524
Test RMSE: 0.001819384722820225
Test MEAN: 1.0004620299768485
Test VOL: 0.00970354566651319

Equal Weights Model Results for model 2:
Validation RMSE: 0.001869762846579279
Test RMSE: 0.002025564603282767
Test MEAN: 1.000323305560954
Test VOL: 0.01008384404038763

Equal Weights Model Results for model 3:
Validation RMSE: 0.0018409818530646968
Test RMSE: 0.0019061054058033676
Test MEAN: 1.0009011627521072
Test VOL: 0.007906547870839895

Equal Weights Model Results for model 4:
Validation RMSE: 0.002047125856897845
Test RMSE: 0.0017836410912025976
Test MEAN: 1.0010289584726735
Test VOL: 0.00637445295648171

Equal Weights Model Results for model 5:
Validation RMSE: 0.002071094081030228
Test RMSE: 0.0022252515108796625
Test MEAN: 1.0001241902175797
Test VOL: 0.007975533940201873

Equal Weights Model Results for model 6:
Validation RMSE: 0.0017935612365303213
Test RMSE: 0.0023468240981766447
Test MEAN: 0.99999103

In [35]:
# print test results
print(f'Models test results with rebalancing period of {rbp} month(s) are: ')
deep_temp = pd.DataFrame(deep_nnf_test_results)
deep_temp = deep_temp.iloc[deep_best_result_index]
shallow_temp = pd.DataFrame(shallow_nnf_test_results)
shallow_temp = shallow_temp.iloc[deep_best_result_index]
equal_w_temp = pd.DataFrame(equal_w_model_test_results)
equal_w_temp = equal_w_temp.iloc[deep_best_result_index]

sp_temp_rmse = '-'
sp_temp_mean = daily_return(date_slicer(df_sp, '2018-01-01', 6, deep_best_result_index)).mean()[0]
sp_temp_std = daily_return(date_slicer(df_sp, '2018-01-01', 6, deep_best_result_index)).std()[0]
sp_temp = pd.DataFrame([sp_temp_rmse, sp_temp_mean, sp_temp_std], index=deep_temp.index)

final_result = pd.concat([deep_temp, shallow_temp, equal_w_temp], axis=1, join='inner')
final_result.columns = ['Deep NNF', 'Shallow NNF', '1/N Model']
final_result

Models test results with rebalancing period of 1 month(s) are: 


,Deep NNF,Shallow NNF,1/N Model
RMSE,0.001980,0.012616,0.002026
MEAN,1.000310,1.000618,1.000323
VOL,0.010074,0.014507,0.010084
